<a href="https://colab.research.google.com/github/10Dennisw/translation-from-english-to-dutch/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing libraries
!pip install transformers torch sentencepiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00


In [ ]:
# Importing modules
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
# Accessing drive to import data later
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Corpus/

Mounted at /content/drive
/content/drive/My Drive/Corpus


In [ ]:
# Opening English file and reading the sentences
text_file = "europarl-v7.nl-en(english).txt"
with open(text_file) as f:
    lines_english = f.read().split("\n")[:-1]

# creating a text lines for English
text_eng = []
for line in lines_english:
    english = line.split("\t")
    text_eng.append(english)

# Opening Dutch file and reading the sentences
text_file = "europarl-v7.nl-en(nl).txt"
with open(text_file) as f:
    lines_dutch = f.read().split("\n")[:-1]

# creating a list for lines in Dutch
text_nl = []
for line in lines_dutch:
    dutch = line.split("\t")
    text_nl.append(dutch)

# Cutting the list due to resource constraints
new_text_eng = []
new_text_nl = []

# selecting sentences which are 15 words or less
# when 10,000 sentences are extracted, the loop broken
for i in range(len(text_eng)):
  for j in text_eng[i]:
    eng = j
  eng = eng.split(" ")
  if len(eng) >= 1 and len(eng) <= 15:
    new_text_eng.append(text_eng[i])
    new_text_nl.append(text_nl[i])
  if len(new_text_eng) == 100000:
    break

# Pairing the sentences together
text_pairs = []
for i in range(100000):
  english_sent = str(new_text_eng[i])
  dutch_sent = str(new_text_nl[i])
  text_pairs.append((english_sent, dutch_sent))

In [ ]:
import random

# Creating the Training, validation and testing pairs
random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs)) #15%
num_train_samples = len(text_pairs) - 2 * num_val_samples #70%

train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [ ]:
def generate_tokens(tokenizer, text_pairs):
  '''
  A function to tokenise the text_pairs and return the tokenised data
  tokenizer: is the tokenizer imported that belongs to the model
  text pairs: is the text pairs. This will be two sentences. One in English, one in dutch.
  tokenised_data: the function will return the tokenised_data
  '''

  # Extractng the sentences from the text pairs
  english_texts = [pair[0][2:-2] for pair in text_pairs]  # Removing square brackets
  dutch_texts = [pair[1][2:-2] for pair in text_pairs]   # Removing square brackets

  # Tokenizing the text pairs
  inputs = tokenizer(english_texts, return_tensors="pt", padding=True, truncation=True, max_length=30) #inputs are tokenized english sentences and attention mask
  labels = tokenizer(dutch_texts, return_tensors="pt", padding=True, truncation=True, max_length=30).input_ids #labels are tokenized dutch sentences

  # Generating the tokenised data, including labels (tokenised dutch sentence)
  tokenized_data = {
      "input_ids": inputs["input_ids"],
      "attention_mask": inputs["attention_mask"],
      "labels": labels
  }

  return tokenized_data  # returning the tokenised data, including input, atttention mask and labels

In [ ]:
# Load the tokenizer associated with your pretrained model
model_name = "Helsinki-NLP/opus-mt-en-nl"
tokenizer = AutoTokenizer.from_pretrained(model_name)


training_tokenized_data = generate_tokens(tokenizer, train_pairs)
eval_tokenized_data = generate_tokens(tokenizer, val_pairs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# Defining the model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/316M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import Dataset

# Defining function to create dictionary
class TranslationDataset(Dataset):
    # initisialisation of input_ids, attention mask and labels
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data['input_ids']
        self.attention_mask = tokenized_data['attention_mask']
        self.labels = tokenized_data['labels']

    # Getting the number of steps
    # This needs to be known for training (learning rate scheduler)
    def __len__(self):
        return len(self.input_ids)

    # returns a a dictionairy of the input_ids, attention mask and labels
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx],
        }


In [ ]:
import torch
# Using fucntion to generating Train and Evaluation Datasets
train_dataset = TranslationDataset(training_tokenized_data)
eval_dataset = TranslationDataset(eval_tokenized_data)

In [ ]:
!pip install --upgrade accelerate

In [ ]:
# Defining training arguments
batch_size = 128
training_args_dict = {
    "output_dir": f"{model_name}-finetuned-english-to-nl",
    "evaluation_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": batch_size,
    "per_device_eval_batch_size": batch_size,
    "weight_decay": 0.01,
    "save_total_limit": 3,
    "num_train_epochs": 10,
    "predict_with_generate": True,
    "push_to_hub": False,
}

# Defining training arguments for the Seq2Seq problem
args = Seq2SeqTrainingArguments(**training_args_dict)

In [ ]:
# Creating a Seq2SeqTrainer instance and specifying training parameter
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    )

# Setting model to training mode
model.train()

# Training the model on the corpus
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.656500,1.085754
2,1.083100,0.963011
3,0.944000,0.912225
4,0.866600,0.888980
5,0.804700,0.876008
6,0.765700,0.868400
7,0.732200,0.866742
8,0.703300,0.864638
9,0.685600,0.864611
10,0.668400,0.864209


Checkpoint destination directory Helsinki-NLP/opus-mt-en-nl-finetuned-english-to-nl/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[67027]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[67027]], 'forced_eos_t

TrainOutput(global_step=5470, training_loss=0.8709478681658478, metrics={'train_runtime': 3773.4181, 'train_samples_per_second': 185.508, 'train_steps_per_second': 1.45, 'total_flos': 5561450496000000.0, 'train_loss': 0.8709478681658478, 'epoch': 10.0})

In [ ]:
# Setting device to Cuda to make sure that tensor input is move to GPU
# Neccesary to run on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def translate(text, model, tokenizer):
  '''
  inputs: text: source texts
  model: the fine-tuned model
  tokenizer: the tokenizer used to tokenize the source texts, corresponding to the model
  output: the decoded translated text
  '''
  inputs = tokenizer(text, return_tensors="pt", max_length=30, truncation=True) # tokenizing the input sentence
  inputs = {key: tensor.to(device) for key, tensor in inputs.items()}  # Move inputs to the cuda/ GPU device

  # Setting the model in evaluation mode
  model.eval()
  # Generating translation in the form of tokens using fine-tuned model
  outputs = model.generate(inputs['input_ids'], max_length=30, num_beams=4, early_stopping=True)
  # Decoding the outputs to generate a sentence
  translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return translated_text # Returning the decoded translation


In [ ]:
# Creating an index list using the random library in the test pair
indx_lst = []
for i in range(10):
  indx_lst.append(random.randint(0, len(test_pairs)-1))

# Printing the initial English Sentence, Translated English Sentence and Actual Dutch Sentence according to corpus
for i in indx_lst:
  source_text = test_pairs[i][0][2:-2] # Extracting English sentence
  translated_text = translate(source_text, model, tokenizer) # Getting Dutch Tranlsation using function translate defined earlier
  actual_translation = test_pairs[i][1][2:-2] # Extracting Dutch translation according to corpus

  # Printing the source, translated and actual translation, which are defined above
  print("--> Source Text:", source_text)
  print("--> Translation:", translated_text)
  print("--> Translation (according to corpus):", actual_translation)

  # Printing a space after to make it easy to identify source, translated and actual translation
  if i == 9:
    break
  else:
    print()

--> Source Text: The citizens need time to make provision for their old age.
--> Translation: De burgers hebben tijd nodig om voor hun ouderdom te zorgen.
--> Translation (according to corpus): De burgers hebben tijd nodig om zich in te stellen op de ouderdom, en daarom moeten wij nu handelen.

--> Source Text: (The President cut the speaker off)
--> Translation: (Spreker wordt door de Voorzitter onderbroken)
--> Translation (according to corpus): (Spreker wordt door de Voorzitter onderbroken)

--> Source Text: Why not?
--> Translation: Waarom niet?
--> Translation (according to corpus): Waarom niet?

--> Source Text: But if it can only be done by codecision we will have a delay.
--> Translation: Als dit echter alleen via medebeslissing kan gebeuren, komt er e
--> Translation (according to corpus): Als dat echter uitsluitend via een medebeslissingsprocedure kan gebeuren, zullen er vertragingen ontstaan.

--> Source Text: I welcome the clear orientation of the report to planning for enl

In [ ]:
total_size = 0
for param in model.parameters():
    # Param is a torch.Tensor
    total_size += param.nelement() * param.element_size()
for buffer in model.buffers():
    # Buffer is a torch.Tensor
    total_size += buffer.nelement() * buffer.element_size()

size_mb = total_size / (1024 * 1024)  # Convert from bytes to megabytes

print(f"Approximate model size: {size_mb:.2f} MB")

Approximate model size: 301.54 MB


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/saved_models/translator_model.pth')